# The lonely machine

In [ ]:
# @markdown Initialize

!git clone https://github.com/Kostis-S-Z/lonelymachine.git
%cd lonelymachine
!pip install -r requirements.txt

In [ ]:
# @markdown This Machine uses the Google Street View API. In order to get it started, you need to provide your API key and secret. You can get them from the [Google Cloud Console](https://console.cloud.google.com/google/maps-apis/).

# ===== API Credentials =====
from getpass import getpass
api_key = getpass("Paste your Street View API Key here:")
api_secret = getpass("Paste your Street View API Key here:")

In [ ]:
# @title ## Select your area of exploration { run: "auto" }

# @markdown 1.   Visit https://boundingbox.klokantech.com

# @markdown 2.   In the bottom left corner there is a dropdown menu which defines the format of the coordinates. Select `DublinCore`

# @markdown 3.   Move to your area of interest and draw the box that you like

# @markdown 4.   In the bottom left corner (`DCMI BOX`) you will have your 4 coordinates. Click on the icon right of the text box to copy them.

# @markdown 5.   Input the coordinates at the box below (Note that west-east and north-south should have same amount of decimal precision)


# @markdown ## Input the coordinates you selected from the website

# ===== Bounding Box and Generation Parameters =====
min_lat = 37946894  # @param {type:"number"}
max_lat = 38092677  # @param {type:"number"}
min_lon = 23665374  # @param {type:"number"}
max_lon = 23926643  # @param {type:"number"}


# ## Select sampling method
# Choose between sampling photos from evenly spaced points within the area, or let the *machine* choose on its "own"
method = "random"  # @param ["random", "even"]

# @markdown ## Select number of photos you want to generate
n_locs = 5  # @param {type:"slider", min:1, max:20, step:1}

In [ ]:
# @title Advanced parametrization { run: "auto" }

# @markdown Dimensions in pixels of photo
photo_size = "640x320"  # @param ["640x640", "640x320", "320x640", "320x320"]

# @markdown Horizontal field of view
fov = 120  # @param {type:"slider", min:0, max:120, step:5}

# @markdown Compass heading of the camera, e.g. North is 0 or 360. East is 90. South is 180. West is 270.
compass = 90  # @param {type:"slider", min:0, max:360, step:10}

# @markdown Sometimes the exact point will not have a photo, define the slack in radius meters to allow the machine to search
radius = 55  # @param {type:"slider", min:5, max:500, step:5}

# @markdown Specifies the up or down angle of the camera relative to the Street View vehicle
pitch = 10  # @param {type:"slider", min:-90, max:90, step:10}

## Run code

In [ ]:
import os
from source.the_machine.locations.generator import create_locations
from source.the_machine.api.client import fetch_images

# Build generation parameters dictionary for create_locations
gen_params = {
    "min_lat": int(min_lat),
    "max_lat": int(max_lat),
    "min_lon": int(min_lon),
    "max_lon": int(max_lon),
    "method": method,
    "n_locs": int(n_locs),
}
print("Generating location CSV file with parameters:")
print(gen_params)
file_path = create_locations(gen_params)
print("CSV file created:", file_path)

# Set the API credentials as environment variables (used by fetch_images)
os.environ["STREET_VIEW_API_KEY"] = api_key
os.environ["STREET_VIEW_API_SECRET"] = api_secret

# Build photo request parameters dictionary
photo_params = {
    "n_addresses": int(n_locs),
    "size": photo_size,
    "fov": fov,
    "heading": compass,
    "radius": str(radius),
    "pitch": pitch,
    "source": "default",
    "return_error_code": "true",
}
print("\nPhoto request parameters:")
print(photo_params)

# Fetch images based on the generated CSV and provided parameters
fetch_images(input_file=file_path, params=photo_params, manual_check=False)

## Execute

In [ ]:
# @title Tell bot to go to a specific point

from source.the_machine.api.client import fetch_image_from_location

# Mandatory format 'XX.XXXXX,XX.XXXXX', e.g '38.059576,23.737887'
location = "38.059576,23.737887"  # @param {type: "string"}
filename = f"my_{location}.jpg"

fetch_image_from_location(location, api_key, api_secret, photo_params, filename)

from IPython.display import Image
Image(filename)

### Notes

- Street View imagery comes from two sources: Google (non-public developer-controlled imagery), and the public (UGC, or public user-generated content). Note that there is no way to use only one source of Street View imagery over the other. Learn more about our Street view imagery privacy policy.